In [1]:
#Analysis of Temperature data at one particular depth over time in Alaska
#By: Ksenia Lepikhina
#Data: http://gtnpdatabase.org/boreholes in the US with data: yes
#Not completed

import numpy as np
import matplotlib.pyplot as plt
import math
import pandas as pd
import csv
import glob
import shutil
import os
from itertools import chain
import fnmatch
import subprocess
from datetime import datetime, date
from pathlib import Path
import re
from operator import is_not


#create df to store summary of contiunous data
summaryDataFrameCont = pd.DataFrame(data=None, index=None, columns=["Location", "Lat", "Long", "D/C", "Rows", "Cols" ,"TotFields","NotNA", "PercComplete","MinD", "MaxD"], dtype=None, copy=False)


#continuous zone
countContFiles = 0 #initialize counter for number of csv
os.chdir("/Users/Ksenia/Desktop/EarthLabPython/TimeSeries/ContinuousTimeSeries/") #directory for continuous

#create lists
locationsCont = []
#locationNamesListCont = []
latCont = []
longCont = []
DCcont = []
rowCountCont = []
colCountCont = []
totalFieldCountCont = []
NAcountCont = []
minDepthCont = []
maxDepthCont = []
completenessCont = []

for contFile in glob.glob("*.csv"):
    #just looking at ground data for now
    if "Ground" not in contFile: 
        #move from one directory to another if not ground data
        originalDirUnneededCont = "/Users/Ksenia/Desktop/EarthLabPython/TimeSeries/ContinuousTimeSeries/%s" % contFile
        destinationDirUnneededCont = "/Users/Ksenia/Desktop/EarthLabPython/TimeSeries/ContinuousTimeSeries/NotGroundData/%s" % contFile
        shutil.move(originalDirUnneededCont, destinationDirUnneededCont)
    else:
        countContFiles+=1
        locationsCont.append(contFile)
        DCcont.append("C")
        
        #count rows
        row_countCont = sum(1 for row in csv.reader(open(contFile))) - 1
        #print(row_countCont)
        rowCountCont.append(row_countCont)
        
        #count columns
        readerCont = csv.reader(open(contFile))
        colCont = len(next(readerCont)) - 1
        colCountCont.append(colCont)
        
        #calculate number of fields
        rowArrayCont = np.array(row_countCont)
        colArrayCont = np.array(colCont)     
        totFieldsCont = rowArrayCont * colArrayCont
        #print(contFile, totFieldsCont)
        totalFieldCountCont.append(totFieldsCont)
        
        #calculate NA
        countNACont = 0    
        for row in readerCont:
            for field in row:
                if field == "-999":
                    countNACont+=1
        notNACont = totFieldsCont - countNACont 
        NAcountCont.append(notNACont) #append to list
        
        #calculate completeness
        completeCont = (notNACont / totFieldsCont) * 100
        completenessCont.append(completeCont)
        
        indexCont = contFile.index('-Dataset')
        #locationNamesStrCont = contFile[0:indexCont]
        #locationNamesListCont.append(locationNamesStrCont)

        #extract latitude/longitude
        metadataFileCont = contFile[0:indexCont] + '.metadata.txt'
        with open(metadataFileCont, "r") as getDataContTxt:
            linesCont=getDataContTxt.readlines() #read txt
            longContFromFile = linesCont[10] #find lines for lat and long
            latContFromFile = linesCont[11]
            longContData = float(longContFromFile[10:-1]) #extract lat and long
            latContData = float(latContFromFile[10:-1])
            longCont.append(longContData) #append to list
            latCont.append(latContData)
        
        #Find minimum depths in each data set (min column name)
        anotherReaderCont = csv.DictReader(open(contFile))
        headersCont = anotherReaderCont.fieldnames
        minDepthCont.append(headersCont[1])
        
        #Find maximum depths in each data set (max column name)
        maxDepthCont.append(headersCont[-1])

#add to data frame
summaryDataFrameCont["Location"] = sorted(locationsCont)
summaryDataFrameCont["Lat"] = latCont
summaryDataFrameCont["Long"] = longCont
summaryDataFrameCont["D/C"] = DCcont
summaryDataFrameCont["Rows"] = rowCountCont
summaryDataFrameCont["Cols"] = colCountCont
summaryDataFrameCont["TotFields"] = totalFieldCountCont
summaryDataFrameCont["NotNA"] = NAcountCont
summaryDataFrameCont["MinD"] = minDepthCont
summaryDataFrameCont["MaxD"] = maxDepthCont
summaryDataFrameCont["PercComplete"] = completenessCont


#output
print(countContFiles)
summaryDataFrameCont


SyntaxError: EOL while scanning string literal (<ipython-input-1-24377ec729d7>, line 22)

In [2]:
#create df to store summary of discontiunous data
summaryDataFrameDisc = pd.DataFrame(data=None, index=None, columns=["Location", "Lat", "Long", "D/C", "Rows", "Cols","TotFields","NotNA", "PercComplete","MinD", "MaxD"], dtype=None, copy=False)

#discontinuous zone
countDiscontFiles = 0 #initialize counter for number of csv
os.chdir("/Users/Ksenia/Desktop/EarthLabPython/TimeSeries/DiscontinuousTimeSeries/") #directory for discontinuous

#create lists
locationsDisc = []
#locationNamesListDisc = []
latDisc = []
longDisc = []
DCdisc = []
rowCountDisc = []
colCountDisc = []
totalFieldCountDisc = []
NAcountDisc = []
minDepthDisc = []
maxDepthDisc = []
completenessDisc = []

for discFile in glob.glob("*.csv"):
    #just looking at ground data for now
    if "Ground" not in discFile:
        #move from one directory to another if not ground data
        originalDirUnneededDisc = "/Users/Ksenia/Desktop/EarthLabPython/TimeSeries/DiscontinuousTimeSeries/%s" % discFile
        destinationDirUnneededDisc = "/Users/Ksenia/Desktop/EarthLabPython/TimeSeries/DiscontinuousTimeSeries/NotGroundData/%s" % discFile
        shutil.move(originalDirUnneededDisc, destinationDirUnneededDisc)
    else:
        countDiscontFiles+=1 
        locationsDisc.append(discFile)
        DCdisc.append("D")
        
        #number of rows
        row_countDisc = sum(1 for row in csv.reader(open(discFile))) - 1
        rowCountDisc.append(row_countDisc)
        
        #number of columns
        readerDisc = csv.reader(open(discFile))
        colDisc = len(next(readerDisc)) - 1
        colCountDisc.append(colDisc)
        
        
        #calculate number of fields
        rowArrayDisc = np.array(row_countDisc)
        colArrayDisc = np.array(colDisc)     
        totFieldsDisc = rowArrayDisc * colArrayDisc
        #print(discFile, totFields)
        totalFieldCountDisc.append(totFieldsDisc)
        
        #calculate not NA
        countNADisc = 0    
        for row in readerDisc:
            for field in row:
                if field == "-999":
                    countNADisc+=1
        notNADisc = totFieldsDisc - countNADisc 
        NAcountDisc.append(notNADisc) #append to list
        
        #calculate completeness
        completeDisc = (notNADisc / totFieldsDisc) * 100
        completenessDisc.append(completeDisc)
        
        indexDisc = discFile.index('-Dataset') 
        #locationNamesStrDisc = discFile[0:indexDisc] <- for removing long file name
        #locationNamesListDisc.append(locationNamesStrDisc)    

        #extract latitude/longitude from txt
        metadataFileDisc = discFile[0:indexDisc] + '.metadata.txt'
        with open(metadataFileDisc, "r") as getDataDiscTxt:
            linesDisc=getDataDiscTxt.readlines() #read txt
            longDiscFromFile = linesDisc[10] #line 10 has longitude (kind of hard coded oops)
            latDiscFromFile = linesDisc[11] #line 11 has latitude (kind of hard coded oops)
            longDiscData = float(longDiscFromFile[10:-1]) #extract longitutde number 
            latDiscData = float(latDiscFromFile[10:-1]) #extract latitude number
            longDisc.append(longDiscData) #append to list for data frame
            latDisc.append(latDiscData)
        
        #Find minimum depths in each data set (min column name)
        anotherReaderDisc = csv.DictReader(open(discFile))
        headersDisc = anotherReaderDisc.fieldnames
        minDepthDisc.append(headersDisc[1])
        
        #Find maximum depths in each data set (max column name)
        maxDepthDisc.append(headersDisc[-1])
        

#Populate data frame
summaryDataFrameDisc["Location"] = sorted(locationsDisc)
summaryDataFrameDisc["Lat"] = latDisc
summaryDataFrameDisc["Long"] = longDisc
summaryDataFrameDisc["D/C"] = DCdisc
summaryDataFrameDisc["Rows"] = rowCountDisc
summaryDataFrameDisc["Cols"] = colCountDisc
summaryDataFrameDisc["TotFields"] = totalFieldCountDisc
summaryDataFrameDisc["NotNA"] = NAcountDisc
summaryDataFrameDisc["MinD"] = minDepthDisc
summaryDataFrameDisc["MaxD"] = maxDepthDisc
summaryDataFrameDisc["PercComplete"] = completenessDisc


#output
print(countDiscontFiles)
summaryDataFrameDisc

25


,Location,Lat,Long,D/C,Rows,Cols,TotFields,NotNA,PercComplete,MinD,MaxD
0,Borehole_1110-Bonanza_Creek_2-Dataset_1366-Con...,64.715000,-148.288500,D,1145,9,10305,10273,99.689471,0.01,1.07
1,Borehole_1190-Mary____s_Igloo_East-Dataset_153...,65.106480,-164.698780,D,15036,6,90216,52080,57.728119,0.02,1.9
2,Borehole_1191-Salmon_Lake-Dataset_1537-Continu...,64.908140,-165.053850,D,24196,6,145176,70360,48.465311,0.02,2.8
3,Borehole_610-Fox-Dataset_1139-Constant_Over_In...,64.950610,-147.617694,D,2254,55,123970,22600,18.230217,-0.32,1.06
4,Borehole_615-GI_1-Dataset_1114-Constant_Over_I...,64.875266,-147.846499,D,3,23,69,53,76.811594,2,35.7
5,Borehole_616-GI_2-Dataset_1115-Constant_Over_I...,64.874494,-147.848247,D,3,25,75,66,88.000000,4,52
6,Borehole_618-Smith_Lake_3-Dataset_1145-Constan...,64.867517,-147.858833,D,2573,19,48887,38531,78.816454,0.01,2.96
7,Borehole_619-Smith_Lake_4-Dataset_1146-Constan...,64.866900,-147.858383,D,2572,46,118312,33137,28.008148,-0.15,2.8
8,Borehole_620-Smith_Lake_2-Dataset_1144-Constan...,64.866117,-147.856767,D,2588,40,103520,36178,34.947836,-0.2,3
9,Borehole_621-UAF_Farm_3-Dataset_1116-Constant_...,64.852910,-147.857840,D,8,42,336,148,44.047619,0,20


In [3]:
#merge continuous and discontinuous data frames
contAndDiscDF = pd.merge(summaryDataFrameCont, summaryDataFrameDisc, how='outer') #FIX

#Sort merged dataframe
#contAndDiscDFSorted = contAndDiscDF.sort_values(by='Location')

#Save to csv
contAndDiscCSV = Path("/Users/Ksenia/Desktop/EarthLabPython/contAndDiscDF.csv")
if not contAndDiscCSV.is_file():
    contAndDiscDF.to_csv('/Users/Ksenia/Desktop/EarthLabPython/contAndDiscDF.csv')

#print DF
contAndDiscDF

,Location,Lat,Long,D/C,Rows,Cols,TotFields,NotNA,PercComplete,MinD,MaxD
0,Borehole_102-Franklin_Bluffs_1__deep_-Dataset_...,69.674138,-148.720761,C,8,81,648,220,33.950617,4,58
1,Borehole_103-Franklin_Bluffs__dry__b-Dataset_3...,69.674140,-148.720766,C,2272,18,40896,23941,58.541178,-0.05,0.99
2,Borehole_103-Franklin_Bluffs__dry__b-Dataset_6...,69.674140,-148.720766,C,1536,44,67584,16882,24.979285,-0.11,0.98
3,Borehole_104-Franklin_Bluffs__dry__be-Dataset_...,69.674140,-148.720766,C,1107,8,8856,8847,99.898374,0.09,0.8
4,Borehole_105-Franklin_Bluffs__dry__ib-Dataset_...,69.674140,-148.720766,C,2698,56,151088,40751,26.971699,-0.12,0.97
5,Borehole_106-Franklin_Bluffs__surface_-Dataset...,69.655130,-148.722016,C,4432,75,332400,48747,14.665162,-0.28,0.98
6,Borehole_107-Franklin_Bluffs__wet__b-Dataset_6...,69.655130,-148.722016,C,1560,15,23400,13185,56.346154,-0.13,0.91
7,Borehole_109-Happy_Valley_1__deep_-Dataset_667...,69.156459,-148.837404,C,7,39,273,132,48.351648,3,39.1
8,Borehole_110-Happy_Valley_1_ib-Dataset_1443-Co...,69.146641,-148.848301,C,1548,35,54180,20121,37.137320,-0.13,0.94
9,Borehole_115-Kaktovik__ANWR___Barter_-Dataset_...,70.116530,-143.628550,C,3,35,105,68,64.761905,3,74


In [4]:
#Collected = annually, monthly, weekly, daily, randomly?
timeDataFrame = pd.DataFrame(data=None, index=None, columns=["Location", "D/C","StartYear", "StartMon", "EndYear", "EndMon", "TotYears", "Collected"], dtype=None, copy=False)

#create lists
startTimeList = []
endTimeList = []
collectedList = []
#start date list
SyearList = []
SmonthList = []
SdayList = []
ShourList = []
SminuteList = []
SsecondList = []
#end date list
EyearList = []
EmonthList = []
EdayList = []
EhourList = []
EminuteList = []
EsecondList = []
TotalYearsList = []

#read from both directories and concatenate
filesInCont = glob.glob('/Users/Ksenia/Desktop/EarthLabPython/TimeSeries/ContinuousTimeSeries/*.csv')
filesInDisc = glob.glob('/Users/Ksenia/Desktop/EarthLabPython/TimeSeries/DiscontinuousTimeSeries/*.csv')
files = filesInCont + filesInDisc

count = 0

#iterate through files
for file in files:
    count+=1 #Used to check: 73 files = good 
    
    #open the file
    with open(file) as f:
        reader = csv.reader(f)
        j = next(reader) #skip the header row
        
        #first row
        for i in range(1):
            startTime = reader.__next__()
            startTimeList.append(startTime[0]) #append to list first value in row
            
            #parse start time
            dateTimeStart = re.split('-| |:', startTime[0])
            SyearList.append(int(dateTimeStart[0]))
            SmonthList.append(int(dateTimeStart[1]))
            SdayList.append(int(dateTimeStart[2]))
            ShourList.append(int(dateTimeStart[3]))
            SminuteList.append(int(dateTimeStart[4]))
            SsecondList.append(int(dateTimeStart[-1]))
            
        #last row    
        last_line = subprocess.check_output(["tail", "-1", file])
        endTime = last_line.decode("utf-8") #convert from binary to string
        endTimeList.append(endTime[0:19]) #get full end time
        
        #parse end time
        #dateTimeEnd = re.split('-| |:', last_line[0])
        EyearList.append(int(endTime[0:4]))
        EmonthList.append(int(endTime[5:7]))
        EdayList.append(int(endTime[8:10]))
        EhourList.append(int(endTime[11:13]))
        EminuteList.append(int(endTime[14:16]))
        EsecondList.append(int(endTime[17:19]))
        
        #total year
        difference = int(endTime[0:4]) - int(dateTimeStart[0])
        TotalYearsList.append(difference)
    
    #Shorten file name search
    indexStart = file.index('-')
    indexEnd = file.index('-Ground')
    collected = file[indexStart:indexEnd]
    
    #Frequency of collection
    if "2_hours" in collected:
        collectedList.append("Every 2 Hours")
        #print("Every 2 hours")
    elif "Hourly" in collected:
        collectedList.append("Hourly")
        #print("Hourly")
    elif "Daily" in collected:
        collectedList.append("Daily")
        #print("Daily")
    elif collected == "Weekly":
        collectedList.append("Weekly")
        #print("Weekly")    
    elif "Monthly" in collected:
        collectedList.append("Monthly")
        #print("Monthly")
    elif "Yearly" or "Annually" in collected:
        collectedList.append("Yearly")
        #print("Yearly")
    else:
        collectedList.append("Other")
        #print("Other")
                   
#Populate data frame
timeDataFrame["Location"] = contAndDiscDF["Location"]
timeDataFrame["D/C"] = contAndDiscDF["D/C"]
timeDataFrame["StartYear"] = SyearList
timeDataFrame["StartMon"] = SmonthList
# timeDataFrame["StartDay"] = SdayList
# timeDataFrame["StartHour"] = ShourList
# timeDataFrame["StartMin"] = SminuteList
# timeDataFrame["StartSec"] = SsecondList
timeDataFrame["EndYear"] = EyearList
timeDataFrame["EndMon"] = EmonthList
timeDataFrame["TotYears"] = TotalYearsList
# timeDataFrame["EndDay"] = EdayList
# timeDataFrame["EndHour"] = EhourList
# timeDataFrame["EndMin"] = EminuteList
# timeDataFrame["EndSec"] = EsecondList
timeDataFrame["Collected"] = collectedList

#save to csv
timeCSV = Path("/Users/Ksenia/Desktop/EarthLabPython/timeDataFrame.csv")
if not timeCSV.is_file():
    timeDataFrame.to_csv('/Users/Ksenia/Desktop/EarthLabPython/timeDataFrame.csv')


#show data frame
timeDataFrame

,Location,D/C,StartYear,StartMon,EndYear,EndMon,TotYears,Collected
0,Borehole_102-Franklin_Bluffs_1__deep_-Dataset_...,C,2005,6,2012,8,7,Yearly
1,Borehole_103-Franklin_Bluffs__dry__b-Dataset_3...,C,2007,8,2013,5,6,Daily
2,Borehole_103-Franklin_Bluffs__dry__b-Dataset_6...,C,2009,6,2013,5,4,Daily
3,Borehole_104-Franklin_Bluffs__dry__be-Dataset_...,C,2006,6,2009,5,3,Daily
4,Borehole_105-Franklin_Bluffs__dry__ib-Dataset_...,C,2006,6,2013,5,7,Daily
5,Borehole_106-Franklin_Bluffs__surface_-Dataset...,C,2001,6,2013,5,12,Daily
6,Borehole_107-Franklin_Bluffs__wet__b-Dataset_6...,C,2006,6,2011,6,5,Daily
7,Borehole_109-Happy_Valley_1__deep_-Dataset_667...,C,2005,6,2012,8,7,Yearly
8,Borehole_110-Happy_Valley_1_ib-Dataset_1443-Co...,C,2007,8,2011,8,4,Daily
9,Borehole_115-Kaktovik__ANWR___Barter_-Dataset_...,C,2008,8,2012,8,4,Yearly


In [5]:
#Merge all dataframes
summaryDFAll = pd.merge(contAndDiscDF, timeDataFrame, on=['Location', 'D/C'])

#save to csv
summaryDFAllCSV = Path("/Users/Ksenia/Desktop/EarthLabPython/summaryDFAll.csv")
if not summaryDFAllCSV.is_file():
    summaryDFAll.to_csv('/Users/Ksenia/Desktop/EarthLabPython/summaryDFAll.csv')

summaryDFAll

,Location,Lat,Long,D/C,Rows,Cols,TotFields,NotNA,PercComplete,MinD,MaxD,StartYear,StartMon,EndYear,EndMon,TotYears,Collected
0,Borehole_102-Franklin_Bluffs_1__deep_-Dataset_...,69.674138,-148.720761,C,8,81,648,220,33.950617,4,58,2005,6,2012,8,7,Yearly
1,Borehole_103-Franklin_Bluffs__dry__b-Dataset_3...,69.674140,-148.720766,C,2272,18,40896,23941,58.541178,-0.05,0.99,2007,8,2013,5,6,Daily
2,Borehole_103-Franklin_Bluffs__dry__b-Dataset_6...,69.674140,-148.720766,C,1536,44,67584,16882,24.979285,-0.11,0.98,2009,6,2013,5,4,Daily
3,Borehole_104-Franklin_Bluffs__dry__be-Dataset_...,69.674140,-148.720766,C,1107,8,8856,8847,99.898374,0.09,0.8,2006,6,2009,5,3,Daily
4,Borehole_105-Franklin_Bluffs__dry__ib-Dataset_...,69.674140,-148.720766,C,2698,56,151088,40751,26.971699,-0.12,0.97,2006,6,2013,5,7,Daily
5,Borehole_106-Franklin_Bluffs__surface_-Dataset...,69.655130,-148.722016,C,4432,75,332400,48747,14.665162,-0.28,0.98,2001,6,2013,5,12,Daily
6,Borehole_107-Franklin_Bluffs__wet__b-Dataset_6...,69.655130,-148.722016,C,1560,15,23400,13185,56.346154,-0.13,0.91,2006,6,2011,6,5,Daily
7,Borehole_109-Happy_Valley_1__deep_-Dataset_667...,69.156459,-148.837404,C,7,39,273,132,48.351648,3,39.1,2005,6,2012,8,7,Yearly
8,Borehole_110-Happy_Valley_1_ib-Dataset_1443-Co...,69.146641,-148.848301,C,1548,35,54180,20121,37.137320,-0.13,0.94,2007,8,2011,8,4,Daily
9,Borehole_115-Kaktovik__ANWR___Barter_-Dataset_...,70.116530,-143.628550,C,3,35,105,68,64.761905,3,74,2008,8,2012,8,4,Yearly


In [6]:
#make new data frame with TotYears>4
summaryDF_withDurBound = summaryDFAll[summaryDFAll.TotYears >=4]


len(summaryDF_withDurBound.index) #50 files

summaryDF_withDurBound

,Location,Lat,Long,D/C,Rows,Cols,TotFields,NotNA,PercComplete,MinD,MaxD,StartYear,StartMon,EndYear,EndMon,TotYears,Collected
0,Borehole_102-Franklin_Bluffs_1__deep_-Dataset_...,69.674138,-148.720761,C,8,81,648,220,33.950617,4,58,2005,6,2012,8,7,Yearly
1,Borehole_103-Franklin_Bluffs__dry__b-Dataset_3...,69.674140,-148.720766,C,2272,18,40896,23941,58.541178,-0.05,0.99,2007,8,2013,5,6,Daily
2,Borehole_103-Franklin_Bluffs__dry__b-Dataset_6...,69.674140,-148.720766,C,1536,44,67584,16882,24.979285,-0.11,0.98,2009,6,2013,5,4,Daily
4,Borehole_105-Franklin_Bluffs__dry__ib-Dataset_...,69.674140,-148.720766,C,2698,56,151088,40751,26.971699,-0.12,0.97,2006,6,2013,5,7,Daily
5,Borehole_106-Franklin_Bluffs__surface_-Dataset...,69.655130,-148.722016,C,4432,75,332400,48747,14.665162,-0.28,0.98,2001,6,2013,5,12,Daily
6,Borehole_107-Franklin_Bluffs__wet__b-Dataset_6...,69.655130,-148.722016,C,1560,15,23400,13185,56.346154,-0.13,0.91,2006,6,2011,6,5,Daily
7,Borehole_109-Happy_Valley_1__deep_-Dataset_667...,69.156459,-148.837404,C,7,39,273,132,48.351648,3,39.1,2005,6,2012,8,7,Yearly
8,Borehole_110-Happy_Valley_1_ib-Dataset_1443-Co...,69.146641,-148.848301,C,1548,35,54180,20121,37.137320,-0.13,0.94,2007,8,2011,8,4,Daily
9,Borehole_115-Kaktovik__ANWR___Barter_-Dataset_...,70.116530,-143.628550,C,3,35,105,68,64.761905,3,74,2008,8,2012,8,4,Yearly
10,Borehole_116-Sagwon_MNT-Dataset_212-Constant_O...,69.433030,-148.673850,C,3764,65,244660,39318,16.070465,-0.28,1.06,2001,6,2011,6,10,Daily


In [7]:
#filter for completeness and how it was collected
#Decided that any data that is more than 40% complete is good
summaryDF_withDur_Compl_Bound = summaryDF_withDurBound[summaryDF_withDurBound.PercComplete >=40]



len(summaryDF_withDur_Compl_Bound.index) #30 files

summaryDF_withDur_Compl_Bound

,Location,Lat,Long,D/C,Rows,Cols,TotFields,NotNA,PercComplete,MinD,MaxD,StartYear,StartMon,EndYear,EndMon,TotYears,Collected
1,Borehole_103-Franklin_Bluffs__dry__b-Dataset_3...,69.674140,-148.720766,C,2272,18,40896,23941,58.541178,-0.05,0.99,2007,8,2013,5,6,Daily
6,Borehole_107-Franklin_Bluffs__wet__b-Dataset_6...,69.655130,-148.722016,C,1560,15,23400,13185,56.346154,-0.13,0.91,2006,6,2011,6,5,Daily
7,Borehole_109-Happy_Valley_1__deep_-Dataset_667...,69.156459,-148.837404,C,7,39,273,132,48.351648,3,39.1,2005,6,2012,8,7,Yearly
9,Borehole_115-Kaktovik__ANWR___Barter_-Dataset_...,70.116530,-143.628550,C,3,35,105,68,64.761905,3,74,2008,8,2012,8,4,Yearly
11,Borehole_117-West_Dock_1__deep_-Dataset_808-Co...,70.374467,-148.552166,C,8,43,344,178,51.744186,2,54.3,2005,8,2012,8,7,Yearly
13,Borehole_118-West_Dock_1__surface_-Dataset_618...,70.374460,-148.552160,C,1511,19,28709,15549,54.160716,0.02,1.07,2009,6,2013,5,4,Daily
23,Borehole_738-Kotzebue_2-Dataset_1124-Constant_...,66.851820,-162.604330,C,3,16,48,40,83.333333,2,29,2007,7,2011,7,4,Yearly
27,Borehole_740-Nome__TBA_-Dataset_1125-Constant_...,64.507960,-165.296150,C,4,32,128,123,96.093750,1,61.8,2007,7,2011,7,4,Yearly
36,Borehole_846-Gakona4-Dataset_1121-Constant_Ove...,62.411500,-145.157416,C,5,46,230,98,42.608696,1,42.46,2008,4,2012,9,4,Yearly
38,Borehole_848-Gakona1-Dataset_1141-Constant_Ove...,62.392916,-145.145277,C,1043,35,36505,18578,50.891659,-0.01,27,2009,7,2013,5,4,Daily


In [8]:
#Filter out files with less than 10 rows (essentially all yearly data)
summaryDF_filtered = summaryDF_withDur_Compl_Bound[summaryDF_withDur_Compl_Bound.Rows >10]

len(summaryDF_filtered.index) #16 files

summaryDF_filtered

,Location,Lat,Long,D/C,Rows,Cols,TotFields,NotNA,PercComplete,MinD,MaxD,StartYear,StartMon,EndYear,EndMon,TotYears,Collected
1,Borehole_103-Franklin_Bluffs__dry__b-Dataset_3...,69.674140,-148.720766,C,2272,18,40896,23941,58.541178,-0.05,0.99,2007,8,2013,5,6,Daily
6,Borehole_107-Franklin_Bluffs__wet__b-Dataset_6...,69.655130,-148.722016,C,1560,15,23400,13185,56.346154,-0.13,0.91,2006,6,2011,6,5,Daily
13,Borehole_118-West_Dock_1__surface_-Dataset_618...,70.374460,-148.552160,C,1511,19,28709,15549,54.160716,0.02,1.07,2009,6,2013,5,4,Daily
38,Borehole_848-Gakona1-Dataset_1141-Constant_Ove...,62.392916,-145.145277,C,1043,35,36505,18578,50.891659,-0.01,27,2009,7,2013,5,4,Daily
39,Borehole_849-Gakona2-Dataset_1140-Constant_Ove...,62.391277,-145.146888,C,1329,19,25251,11942,47.293177,-0.08,1.08,2009,7,2013,5,4,Daily
43,Borehole_93-Howe_Island_1_b-Dataset_1445-Avera...,70.315246,-147.993244,C,1847,22,40634,18565,45.688340,-0.01,1.06,2005,6,2011,6,6,Daily
44,Borehole_94-Howe_Island_1_ib-Dataset_1447-Cons...,70.315152,-147.993158,C,1847,2,3694,3050,82.566324,0.26,0.47,2005,6,2011,6,6,Daily
45,Borehole_94-Howe_Island_1_ib-Dataset_1448-Cons...,70.315152,-147.993158,C,1847,22,40634,18576,45.715411,-0.06,1.02,2005,6,2011,6,6,Daily
46,Borehole_95-Happy_Valley_1_b-Dataset_668-Const...,69.146640,-148.848353,C,1548,24,37152,18173,48.915267,-0.2,0.9,2007,8,2011,8,4,Daily
54,Borehole_618-Smith_Lake_3-Dataset_1145-Constan...,64.867517,-147.858833,D,2573,19,48887,38531,78.816454,0.01,2.96,2006,9,2013,5,7,Daily


In [9]:
fileDepths = pd.DataFrame(data=None, index=None, columns=["Location", "DepthToWorkWith"], dtype=None, copy=False)

depths = []

for file in files:
    #get file that matches Location in summaryDF_filtered data frame
    for k in summaryDF_filtered.Location:
        if k in file:
            #Open file
            with open(file) as f:
                #get header for files
                readerDepths = csv.reader(f)
                header = next(readerDepths)
                headerDepths = header[1:len(header)]
                
                #create list with depth of 0.5 cm?? m??
                val = next((x for x in headerDepths if str(0.5)<=x<str(0.6)), None)
                if val == None:
                    depths.append(val)
                else:
                    depths.append(float(val))
                filteredDepths = list(filter(None, depths))

#Populate data frame
fileDepths["Location"] = summaryDF_filtered["Location"]
fileDepths["DepthToWorkWith"] = depths

#Remove rows with NaN depth in range of 0.5-0.6
fileDepths = fileDepths.dropna()

#Show df
fileDepths

,Location,DepthToWorkWith
1,Borehole_103-Franklin_Bluffs__dry__b-Dataset_3...,0.500
6,Borehole_107-Franklin_Bluffs__wet__b-Dataset_6...,0.560
13,Borehole_118-West_Dock_1__surface_-Dataset_618...,0.525
38,Borehole_848-Gakona1-Dataset_1141-Constant_Ove...,0.500
39,Borehole_849-Gakona2-Dataset_1140-Constant_Ove...,0.520
43,Borehole_93-Howe_Island_1_b-Dataset_1445-Avera...,0.540
45,Borehole_94-Howe_Island_1_ib-Dataset_1448-Cons...,0.510
46,Borehole_95-Happy_Valley_1_b-Dataset_668-Const...,0.550
54,Borehole_618-Smith_Lake_3-Dataset_1145-Constan...,0.500
58,Borehole_621-UAF_Farm_3-Dataset_1143-Constant_...,0.508


In [225]:
#~~ Plan:
# - determine completeness of files at depth to work with 
# - plot each location Temperature vs Time at that depth

In [20]:
atDepthDF = pd.DataFrame(data=None, index=None, columns=["Location", "TotDataAtDepth","NotNAatDepth"], dtype=None, copy=False)

rowCountFiltered = []
notNAFiltered = []
getColFiltered = []

def round_of_rating(number):
    """Round a number to the closest half integer.
    >>> round_of_rating(1.3)
    1.5
    >>> round_of_rating(2.6)
    2.5
    >>> round_of_rating(3.0)
    3.0
    >>> round_of_rating(4.1)
    4.0"""

    return round(number * 2) / 2


for file in files:
    #get file that matches Location in fileDepths data frame
    for k in fileDepths.Location:
        if k in file:
            #Open file
            with open(file) as f:
                #get header for files
                readerFileDepths = csv.reader(f)
                header = next(readerFileDepths)
                headerFileDepths = header[1:len(header)]
                headerFileDepths = list(map(float, headerFileDepths))
                
                #get row count/ total elements in that column
                for i in headerFileDepths:
                    row_countDepth = sum(1 for row in readerFileDepths)
                    if row_countDepth != 0:
                        rowCountFiltered.append(row_countDepth)
                    #print(headerFileDepths, i)

                    
                    
                test = next(obj for obj in headerFileDepths if 0.5<=obj<0.6)
                test = str(test)
                #print(test)
                
                header = header[0:]
                this = header.index(test)
                total = 0
                for row in readerFileDepths:
                    rowNew = row[this]
                    rowNew = int(rowNew)
                    testing = rowNew.isnull().sum()
#                     if row[this] == '-999':
#                         total +=1
                    print(testing)
                #print(total)
                    
                #count NA's
atDepthDF["Location"] = fileDepths["Location"]
atDepthDF["TotDataAtDepth"] = rowCountFiltered
#atDepthDF["NotNAatDepth"] = notNAFiltered

#atDepthDF
